In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so
from gapminder import gapminder

from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from formulaic import Formula

In [3]:
"""
1. Queremos estudiar la relacion entre la longitud de la aleta de un pinguino y el peso del ping¨uino.
Como en una esfera, el peso es proporcional a la longitud del radio elevada al cubo, podemos
conjeturar que un polinomio de grado 3 es apropiado para ajustar el peso en funcion de la
longitud de la aleta. Queremos verificar si nuestra conjetura tiene sustento en los datos.
(a) Datos faltantes. Ejecutar el siguiente codigo y observar si hay filas con datos faltantes
(NaN).
penguins = sns . load_dataset (" penguins ")
penguins . head ()
Para hacerlo en forma mas sistematica (en lugar de mirar solo algunas filas) podes usar
el siguiente codigo
penguins . isnull () . values .any ()
Para eliminar las filas con valores faltantes aplicamos al DataFrame el metodo dropna().
Eliminar las filas con datos faltantes del DataFrame de ping¨uinos y verificar que el
DataFrame resultante no contiene valores faltantes
"""
penguins = sns.load_dataset ("penguins")
print(penguins . isnull () . values .any ())# consulto si tiene lineas NaN
penguins = penguins.dropna() #saque las filas con datos NaN
penguins . isnull () . values .any () # consulto si tiene lineas NaN. Como es False, ya me quedo trank
penguins.head()

True


species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1           18.7              181.0   
1  Adelie  Torgersen            39.5           17.4              186.0   
2  Adelie  Torgersen            40.3           18.0              195.0   
4  Adelie  Torgersen            36.7           19.3              193.0   
5  Adelie  Torgersen            39.3           20.6              190.0   

   body_mass_g     sex  
0       3750.0    Male  
1       3800.0  Female  
2       3250.0  Female  
4       3450.0  Female  
5       3650.0    Male

In [4]:
"""
(b) Dividir el dataset resultante en un grupo de entrenamiento y uno de test (80% - 20%)
"""
penguins_train, penguins_test = train_test_split(penguins, test_size=0.2, random_state=42)
# Mostrar la cantidad de datos en cada conjunto
print("Tamaño del dataset:", len(penguins))
print("Tamaño del conjunto de entrenamiento:", len(penguins_train))
print("Tamaño del conjunto de prueba:", len(penguins_test))

Tamaño del dataset: 333
Tamaño del conjunto de entrenamiento: 266
Tamaño del conjunto de prueba: 67


In [5]:
# Preparar los datos
X_train = penguins_train[['flipper_length_mm']]  # Longitud de la aleta como característica
y_train = penguins_train['body_mass_g']  # Peso del pingüino como variable objetivo

X_test = penguins_test[['flipper_length_mm']]
y_test = penguins_test['body_mass_g']

X = penguins_test[['flipper_length_mm']]
y = penguins_test['body_mass_g']

X_train , X_test , y_train , y_test = train_test_split (X,y, test_size =0.2 , random_state =42)

"""
(c) Crear y ajustar 3 modelos utilizando polinomios de grados 1, 2 y 3.
"""
modelos = []
# Iterar sobre los grados de polinomios
for grado in [1, 2, 3]:
    polynomial_features = PolynomialFeatures(degree=grado)
    X_train_poly = polynomial_features.fit_transform(X_train)
    # print('aa')
    
    modelo = LinearRegression()
    modelo.fit(X_train_poly, y_train)
    
    modelos.append((grado, modelo))
    
    # Calcular el error cuadrático medio en el conjunto de prueba
    X_test_poly = polynomial_features.transform(X_test)
    y_pred = modelo.predict(X_test_poly)
    ecm = mean_squared_error(y_test, y_pred)
    print(f"Grado del polinomio {grado}: ecm = {ecm}")

# Seleccionar el modelo con el menor error cuadrático medio en el conjunto de prueba
best_degree, best_model = min(modelos, key=lambda x: mean_squared_error(y_test, x[1].predict(PolynomialFeatures(degree=x[0]).fit_transform(X_test))))
print(f"El mejor modelo es un polinomio de grado {best_degree}")


Grado del polinomio 1: ecm = 115281.19585481322
Grado del polinomio 2: ecm = 114796.42114744331
Grado del polinomio 3: ecm = 111453.89223148454
El mejor modelo es un polinomio de grado 3


In [6]:
"""
(d) Calcular para cada uno el error prediccion en el grupo de entrenamiento y en el grupo de
test.
"""
# Calcular el error de predicción en el grupo de entrenamiento y en el grupo de prueba para cada modelo
for grado, modelo in modelos:
    # Predicciones en el conjunto de entrenamiento
    y_train_pred = modelo.predict(PolynomialFeatures(degree=grado).fit_transform(X_train))
    ecm_train = mean_squared_error(y_train, y_train_pred)
    
    # Predicciones en el conjunto de prueba
    y_test_pred = modelo.predict(PolynomialFeatures(degree=grado).fit_transform(X_test))
    ecm_test = mean_squared_error(y_test, y_test_pred)
    
    print(f"Grado del polinomio {grado}:")
    print(f"    ecm en el conjunto de entrenamiento: {ecm_train}")
    print(f"    ecm en el conjunto de prueba: {ecm_test}")


Grado del polinomio 1:
    ecm en el conjunto de entrenamiento: 132508.46957248118
    ecm en el conjunto de prueba: 115281.19585481322
Grado del polinomio 2:
    ecm en el conjunto de entrenamiento: 125516.83420887454
    ecm en el conjunto de prueba: 114796.42114744331
Grado del polinomio 3:
    ecm en el conjunto de entrenamiento: 124686.64392550847
    ecm en el conjunto de prueba: 111453.89223148454


In [7]:
"""
(e) ¿Cual modelo tiene el menor error (ECM) en el ajuste? ¿Cual el menor error (ECM) de
prediccion?
"""

# Encontrar el modelo con el menor error cuadrático medio en el ajuste y en la predicción
for grado, modelo in modelos:
    # Predicciones en el conjunto de entrenamiento
    y_train_pred = modelo.predict(PolynomialFeatures(degree=grado).fit_transform(X_train))
    ecm_train = mean_squared_error(y_train, y_train_pred)
    
    # Predicciones en el conjunto de prueba
    y_test_pred = modelo.predict(PolynomialFeatures(degree=grado).fit_transform(X_test))
    ecm_test = mean_squared_error(y_test, y_test_pred)
    
    # Actualizar el modelo con el menor error cuadrático medio en el ajuste
    min_ecm_train = float('inf')
    
    if ecm_train < min_ecm_train:
        min_ecm_train = ecm_train
        best_fit_model = (grado, modelo)
    
    # Actualizar el modelo con el menor error cuadrático medio de predicción
    min_ecm_test = float('inf')
    if ecm_test < min_ecm_test:
        min_ecm_test = ecm_test
        best_pred_model = (grado, modelo)

# Imprimir los resultados
print(f"El modelo con el menor error cuadrático medio en el ajuste es un polinomio de grado {best_fit_model[0]} con ecm = {min_ecm_train}")
print(f"El modelo con el menor error cuadrático medio de predicción es un polinomio de grado {best_pred_model[0]} con ecm = {min_ecm_test}")



El modelo con el menor error cuadrático medio en el ajuste es un polinomio de grado 3 con ecm = 124686.64392550847
El modelo con el menor error cuadrático medio de predicción es un polinomio de grado 3 con ecm = 111453.89223148454


In [8]:
"""
(f) En base a los resultados obtenidos, ¿cual de los tres modelos utilizarıa?
"""
"El 3"


'El 3'

In [9]:
"""
2. En el archivo 50_startups.csv tenemos los siguientes datos de 50 companıas: gastos en
investigacion y desarrollo, gastos administrativos, gastos en marketing y ganancias. Queremos
estimar las ganancias a partir de los gastos en las distintas areas.

(a) Leer el archivo, y realizar un grafico de dispersion para cada par de variables. Se pueden
generar todos los graficos automaticamente con el pairplot.
"""
startups = pd.read_csv('./datos/50_startups.csv')
# sns.pairplot(
#     data= startups, aspect=.8
# )
"""
En base a estos graficos, si quisieramos predecir la ganancia mediante un modelo lineal
utilizando una sola variable predictora, ¿cual variable utilizarıa? Disenar un experimento
para verificar su respuesta
"""

# Paso 1: Cargar los datos
startups = pd.read_csv('./datos/50_startups.csv')

# Paso 1.1: Excluir columnas no numéricas
startups_numeric = startups.select_dtypes(include=['float64', 'int64'])

# Paso 2: Calcular la correlación entre las variables
correlation = startups_numeric.corr()
print("Correlación con las ganancias:")
print(correlation['Ganancia'])

# Paso 3 y 4: Entrenar modelos lineales y evaluar su rendimiento
variables_predictoras = ['Investigacion', 'Administracion', 'Marketing']

for variable in variables_predictoras:
    # Seleccionar la variable como predictor y las ganancias como variable dependiente
    X = startups[[variable]]
    y = startups['Ganancia']
    
    # Dividir los datos en conjunto de entrenamiento y conjunto de prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Entrenar el modelo lineal
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Realizar predicciones en el conjunto de prueba
    y_pred = model.predict(X_test)
    
    # Evaluar el rendimiento del modelo
    mse = mean_squared_error(y_test, y_pred)
    print(f"Variable predictora: {variable}, MSE: {mse}")

# Paso 5: Comparar los resultados

Correlación con las ganancias:
Investigacion     0.972900
Administracion    0.200717
Marketing         0.747766
Ganancia          1.000000
Name: Ganancia, dtype: float64
Variable predictora: Investigacion, MSE: 59510962.80787996
Variable predictora: Administracion, MSE: 964533647.8006195
Variable predictora: Marketing, MSE: 900533964.2328259


In [10]:
"""
(b) En este ejemplo, ¿considera que un modelo lineal multivariado ayudarıa a predecir mejor
la ganancia que el modelo lineal univariado del ıtem anterior? Realizar un experimento
para verificar su respuesta.
"""
display(startups.head())
startups.Estado.unique() #new york, california, florida
startups.isnull().values.any() # consulto si tiene lineas NaN. Como es False, ya me quedo trank

# Crear variables predictoras y variable dependiente
y, X = (
    Formula('Ganancia ~ Marketing + Administracion + Investigacion + Estado')
    .get_model_matrix(startups)
)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y ajustar el modelo lineal
modelo = linear_model.LinearRegression(fit_intercept = False)
modelo.fit(X, y)

# Imprimir coeficientes
print("Coeficientes:", modelo.coef_)

# Evaluar el modelo multivariado
y_pred = modelo.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE del modelo lineal multivariado:", mse)


Investigacion  Administracion  Marketing      Estado   Ganancia
0      165349.20       136897.80  471784.10    New York  192261.83
1      162597.70       151377.59  443898.53  California  191792.06
2      153441.51       101145.55  407934.54     Florida  191050.39
3      144372.41       118671.85  383199.62    New York  182901.99
4      142107.34        91391.77  366168.42     Florida  166187.94

Coeficientes: [[ 5.01253438e+04  2.69798610e-02 -2.70043196e-02  8.06023114e-01
   1.98788793e+02 -4.18870191e+01]]
MSE del modelo lineal multivariado: 66470218.015456036


In [11]:
"""
3. En el Ejercicio 1 no tuvimos en cuenta el sexo del ping¨uino para predecir el peso, y puede
ser una variable importante. Se quiere predecir ahora el peso de un ping¨uino usando como
variables predictoras el largo de la aleta y el sexo del ping¨uino (utilizar el DataFrame sin datos
faltantes, como vimos en el Ejercicio 1 (a)).
"""
penguins = sns.load_dataset ("penguins")
print(penguins . isnull () . values .any ())# consulto si tiene lineas NaN
penguins = penguins.dropna() #saque las filas con datos NaN
penguins . isnull () . values .any () # consulto si tiene lineas NaN. Como es False, ya me quedo trank
penguins.head()


True


species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1           18.7              181.0   
1  Adelie  Torgersen            39.5           17.4              186.0   
2  Adelie  Torgersen            40.3           18.0              195.0   
4  Adelie  Torgersen            36.7           19.3              193.0   
5  Adelie  Torgersen            39.3           20.6              190.0   

   body_mass_g     sex  
0       3750.0    Male  
1       3800.0  Female  
2       3250.0  Female  
4       3450.0  Female  
5       3650.0    Male

In [49]:
"""
(a) ¿Cuales son todos los valores que toma la variable “sex”? ¿Que tipo de variable es:
numerica o categorica, ordinal o nominal? ¿Es una variable binaria?
"""
penguins.sex.unique()
print('Toma variables categoricas nominal (no hay orden inherente ente las categorias macho hembra')
print('Puede hacerse interpretar como una variable binaria')


Toma variables categoricas nominal (no hay orden inherente ente las categorias macho hembra
Puede hacerse interpretar como una variable binaria


In [55]:
"""
(b) Escribir (en lapiz y papel) la ecuacion de un modelo lineal para este caso. ¿Que unidades
tienen las variables y como se codifica la variable “sexo del pinguino”?
"""
print("Un modelo lineal simple con las variables largo de la aleta (numérica) y sexo (categórica) podría escribirse como:    \nPeso =𝛽0 + 𝛽1× Largo de la Aleta + 𝛽2× Sexo    \nDonde:    \n𝛽0: es el intercepto    \n𝛽1: es el coeficiente para el largo de la aleta (en mm),    \n𝛽2: es el coeficiente para el sexo, ajustado dependiendo de la codificación usada (0 o 1 para masculino y femenino, respectivamente, si se usa codificación binaria).")

Un modelo lineal simple con las variables largo de la aleta (numérica) y sexo (categórica) podría escribirse como:    
Peso =𝛽0 + 𝛽1× Largo de la Aleta + 𝛽2× Sexo    
Donde:    
𝛽0: es el intercepto    
𝛽1: es el coeficiente para el largo de la aleta (en mm),    
𝛽2: es el coeficiente para el sexo, ajustado dependiendo de la codificación usada (0 o 1 para masculino y femenino, respectivamente, si se usa codificación binaria).


In [58]:
"""
(c) Codificacion de variables binarias. Si utilizamos Formulaic para generar la matriz de
datos e incluimos la variable sexo en el modelo, automaticamente va a crear una columna
con el sexo codificado con 0’s y 1’s.
Alternativamente, podemos utilizar el siguiente codigo:
"""

from sklearn . preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
sex01 = encoder.fit_transform(penguins[["sex"]])
penguins["sex01"] = sex01

In [59]:
"""
(d) Ajustar el modelo usando todos los datos disponibles. Reportar los coeficientes encontrados y calcular el error de prediccion (ECM).
¿Considera que agregar la variable “sex” mejoro el modelo?
"""

formula = 'body_mass_g ~ flipper_length_mm + sex01'
y, X =  Formula(formula).get_model_matrix(penguins)
y = y.squeeze()

In [48]:
# Separamos en entrenamiento y validación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

# Si no agregamos ninguna opción, el programa va a agregar un nuevo intercept al realizar la regresión.
modelo = linear_model.LinearRegression() 

# Train the model
modelo.fit(X_train, y_train)

# Make predictions
y_pred = modelo.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

print(modelo.intercept_)
print(modelo.coef_[0:4])
display(y,X)

Mean Squared Error: 156294.83
-7601.398101078813
[ 0.          7.89396394 44.08882895 53.3587144 ]


0      3750.0
1      3800.0
2      3250.0
4      3450.0
5      3650.0
        ...  
338    4925.0
340    4850.0
341    5750.0
342    5200.0
343    5400.0
Name: body_mass_g, Length: 333, dtype: float64

Intercept  bill_length_mm  bill_depth_mm  flipper_length_mm
0          1.0            39.1           18.7              181.0
1          1.0            39.5           17.4              186.0
2          1.0            40.3           18.0              195.0
4          1.0            36.7           19.3              193.0
5          1.0            39.3           20.6              190.0
..         ...             ...            ...                ...
338        1.0            47.2           13.7              214.0
340        1.0            46.8           14.3              215.0
341        1.0            50.4           15.7              222.0
342        1.0            45.2           14.8              212.0
343        1.0            49.9           16.1              213.0

[333 rows x 4 columns]